### Imports

In [5]:
import serial
import struct
import subprocess
import time
import sys
from tqdm.notebook import trange, tqdm
!!ls /dev/ttyACM*

['/dev/ttyACM0']

### Setting up environment and commands

In [6]:
try:
    ser = serial.Serial("/dev/ttyACM2")
except:
    ser = serial.Serial("/dev/ttyACM0")


def set_pulse(pulse):
    ser.write(b"\x42" + struct.pack("<I", pulse))
def set_delay(delay):
    ser.write(b"\x41" + struct.pack("<I", delay))
def glitch():
    ser.write(b"\x43")
def powerup():
    ser.write(b"\x44")
def powerdown():
    ser.write(b"\x45")

### Function that tests for succesful debug swd connection

In [7]:
def test_swd():
    try:
        subprocess.check_output(['openocd', '-f', 'interface/stlink.cfg', '-f', 'target/stm32f4x.cfg', '-c', 'init; exit'], stderr=subprocess.STDOUT)
        return True
    except:
        pass
    return False

### Test Powerup of Airtag

In [8]:
powerup()

### Test Powerdown of Airtag

In [9]:
powerdown()

## Run the Glitch!

Adjust the range of Delay accordingly

In [11]:
set_pulse(18)
# Glitch from range 3400 to 4600
# 10 tries for each
set_delay(3400)
# Hit at 614 w/current firmware starting at 3400
# What happens if we start at 3460? --> Hit at 1182
# 3462
[3402, 3438, 3441, 3447]
Hit 4 glitches in total!


In [15]:
hit = False
hits = []
count=0
set_delay(3400)

import time
for delay in range(3400,4700,1):
    for x in range(0,10):
        count +=1
        glitch()
        time.sleep(1.5)
        if(test_swd()):
            hits.append(delay)
            break
print(hits)
foo = len(hits)
print(f"Hit {foo} glitches in total!")

KeyboardInterrupt: 